# Intro
Here i want to see if i can recapitulate the data shown in Cripps 2009, to get an almost ethanologenic strain.

KO's made by Cripps et al:
- ldh
- pfl
- upregulate PDH

I will do two things: 

1) I will insert their KOs, and minimize oxygen to see if i can get the similar fermentation products.

2) I will set the exchange fluxes to what they measure and see if the model still works.

In [1]:
import cameo

In [2]:
import copy
import pandas as pd
import cobra.io
from escher import Builder
from cobra.flux_analysis import flux_variability_analysis
import copy
import numpy as np

In [33]:
model = cobra.io.read_sbml_model('../model/g-thermo.xml')

In [4]:
#make a map of central carbon flux on glucose
solution_glc = model.optimize()
Builder(model = model, map_json = '../map/g-thermo-map.json', reaction_data = solution_glc.fluxes.to_dict())

Builder(reaction_data={'IDPh': 2.14369169735729, 'CAT': 0.0005825847475514569, 'PDHam1hi': 0.0, 'HYDA': 0.0, '…

In [34]:
#make it 'anaerobic'
model.reactions.EX_o2_e.bounds = (-0.1,0)

In [6]:
#make a map of central carbon flux on glucose
solution_o2 = model.optimize()
Builder(model = model, map_json = ../map/g-thermo-map.json', reaction_data = solution_o2.fluxes.to_dict())

Builder(reaction_data={'IDPh': 0.2772120499494164, 'CAT': 0.00011041074320858754, 'PDHam1hi': 0.0, 'HYDA': 0.0…

In [22]:
model.reactions.LDH_L.bounds = (0,0)
model.reactions.PFL.bounds = (0,0)

what I will do is make the change as designated and then compute the optimal growth rate. Then from there, I will fix the optimal growth rate at 99.9% of this value, and compute an FVA to show the confidence intervals from the analysis. 

In [6]:
growth = 0.99*model.optimize().objective_value
model.reactions.biomass.bounds = (growth,growth)

In [7]:
growth

0.08905147361700637

In [8]:
biomass_yld = (growth/10)/0.180156 

In [9]:
biomass_yld

0.04943020139046513

In [10]:
solution = model.optimize()
solution['EX_glc__D_e']

-10.0

In [11]:
y_etoh = -model.optimize()['EX_etoh_e']/model.optimize()['EX_glc__D_e']
y_etoh

0.4625461770013291

In [12]:
y_lac = -model.optimize()['EX_lac__L_e']/model.optimize()['EX_glc__D_e']
y_lac

0.0

In [13]:
y_ac = -model.optimize()['EX_ac_e']/model.optimize()['EX_glc__D_e']
y_ac

0.4269401275485528

In [14]:
y_for = -model.optimize()['EX_for_e']/model.optimize()['EX_glc__D_e']
y_for

0.0

In [15]:
y_pyr = -model.optimize()['EX_pyr_e']/model.optimize()['EX_glc__D_e']
y_pyr

0.0

In [16]:
reactions = model.reactions.EX_glc__D_e, model.reactions.EX_etoh_e,model.reactions.EX_lac__L_e,model.reactions.EX_lac__D_e,model.reactions.EX_ac_e,model.reactions.EX_for_e,model.reactions.EX_pyr_e

In [17]:
model.optimize()

,fluxes,reduced_costs
IDPh,0.220359,0.0
CAT,0.000191,0.0
PDHam1hi,0.000000,0.0
HYDA,0.000000,-0.0
MALHYDRO,0.000000,0.0
...,...,...
C23OSAL,0.000000,0.0
MHMSAH,0.000000,0.0
UREA2tabc,0.000000,0.0
EX_urea_e,0.000000,0.0


In [18]:
with model:
    growth = 0.95*model.optimize().objective_value
    print('growth rate:',growth)
    model_copy = copy.deepcopy(model)
    model_copy.reactions.biomass.bounds = (growth,growth) #fix growth rate at 99.5% of max:
    for rct in reactions:
        print(flux_variability_analysis(model_copy, rct))

growth rate: 0.08459889993615605
             minimum   maximum
EX_glc__D_e    -10.0 -9.487284
           minimum    maximum
EX_etoh_e      0.0  18.928139
             minimum   maximum
EX_lac__L_e      0.0  0.512716
             minimum   maximum
EX_lac__D_e      0.0  0.512716
         minimum   maximum
EX_ac_e      0.0  6.505219
          minimum   maximum
EX_for_e      0.0  0.693765
          minimum   maximum
EX_pyr_e      0.0  2.734484


In [19]:
growth

0.08459889993615605

Next, I will try to fix the exchange rates as that defined by Cripps et al, and see if the model still runs. In doing this, I will set the upper and lower bounds as the measured flux. Ideally we would add the standard deviation as the flexibility in the flux, but as this data is not available from the paper I cannot include it. Instead, I will try with the exact values measured and see if it still runs. If not, then i can include a 5% error margin to the value or so.

In [49]:
model.reactions.EX_etoh_e.bounds = (16.35417, 16.35417)

In [50]:
model.reactions.EX_glc__D_e.bounds = (-10,1000)
model.reactions.EX_ac_e.bounds = (0.67708,0.67708)
model.reactions.EX_lac__L_e.bounds = (0.52083,0.52083)
model.reactions.EX_for_e.bounds = (0,0)
model.reactions.EX_pyr_e.bounds = (0.05208,0.05208)

In [51]:
#block the two KO reactions
model.reactions.LDH_L.bounds = (0,0)
model.reactions.PFL.bounds = (0,0)

In [52]:
#make it 'anaerobic'
model.reactions.EX_o2_e.bounds = (-0.1,0)

In [53]:
growth = model.optimize().objective_value

In [55]:
biomass_yld = (growth/10)/0.180156 
biomass_yld

0.04525364314458203

So it works. It doesn't grow great, but atleast it is possible. The media used in the paper had 2% glucose (20g/l) and 0.5% YE, so the absolute number is hard to compare.

I also just want to try the WT.


In [30]:
model.reactions.EX_etoh_e.bounds = (3.74585, 4.14015)

In [31]:
model.reactions.EX_glc__D_e.bounds = (-10,1000)
model.reactions.EX_ac_e.bounds = (1.62857,1.8)
model.reactions.EX_lac__L_e.bounds = (11.4,12.6)
model.reactions.EX_for_e.bounds = (1.1411,1.56)
model.reactions.EX_pyr_e.bounds = (0,0)

In [32]:
#make it 'anaerobic'
model.reactions.EX_o2_e.bounds = (-0.1,0)

In [33]:
growth = model.optimize().objective_value

In [35]:
biomass_yld = (growth/10)/0.180156 
biomass_yld

0.05049829203695307

Try for just the ldh KO and the PFL, LDH double KO too

In [36]:
model.reactions.EX_etoh_e.bounds = (9.54286, 9.54286)

In [37]:
model.reactions.EX_glc__D_e.bounds = (-10,1000)
model.reactions.EX_ac_e.bounds = (0.28571,0.28571)
model.reactions.EX_lac__L_e.bounds = (0.85714,0.85714)
model.reactions.EX_for_e.bounds = (1.88571,1.88571)
model.reactions.EX_pyr_e.bounds = (3.42857,3.42857)

In [38]:
#KO LDH
model.reactions.LDH_L.bounds = (0,0)

In [39]:
#make it 'anaerobic'
model.reactions.EX_o2_e.bounds = (-0.1,0)

In [40]:
growth = mode
l.optimize().objective_value

In [41]:
biomass_yld = (growth/10)/0.180156 
biomass_yld

0.04198446247032758

Double KO data fitting

In [42]:
model.reactions.EX_etoh_e.bounds = (10.94828,10.94828)

In [43]:
model.reactions.EX_glc__D_e.bounds = (-10,1000)
model.reactions.EX_ac_e.bounds = (0.34483,0.34483)
model.reactions.EX_lac__L_e.bounds = (0.86207,0.86207)
model.reactions.EX_for_e.bounds = (0.00000,0)
model.reactions.EX_pyr_e.bounds = (3.18966,3.18966)

In [44]:
#KO LDH and PFL
model.reactions.LDH_L.bounds = (0,0)
model.reactions.PFL.bounds = (0,0)

In [45]:
#make it 'anaerobic'
model.reactions.EX_o2_e.bounds = (-0.1,0)

In [47]:
growth = model.optimize().objective_value

In [48]:
biomass_yld = (growth/10)/0.180156 
biomass_yld

0.03988478450003583

# Effect of lactate on growth
From the above, we've seen that the main component where the simulation doesn't match the measurements is the lactate production in the WT strian. We also see the rpedicted growth rates are higher than the measured. This can be because the NGAM & GAM are set abased on aerobic data.

But still we want to see the effect of increasing production of lactate, what effect it has on the growth rate. So here I will calculate that for the WT.

In the paper, they get 1.2 mol/mol lactate from glucose, so i will test a range up to 1.5 mol/mol. 

I will use the data from table 5. 

In [100]:
model = cobra.io.read_sbml_model('../model/g-thermo.xml')

In [101]:
#make it 'anaerobic'
model.reactions.EX_o2_e.bounds = (-0.1,0)

In [123]:
#list of lactate exports to try, considering we feed 10 mmol/gcdw/h
lac_con = np.linspace(0,30,51)

In [103]:
#fix all the other exchanges for the WT setting
#because there is no SD given, i will allow 5% deviation from the measurement
model.reactions.EX_etoh_e.bounds = (3.7459, 4.1402)
model.reactions.EX_ac_e.bounds = (1.6286, 1.80)
model.reactions.EX_for_e.bounds = (1.4114, 1.56)
model.reactions.EX_pyr_e.bounds = (0,0)

In [104]:
model.reactions.EX_etoh_e.bounds = (3.74585, 4.14015)

In [105]:
growth_rates = []
yld = []
lac_g_g =[]
for con in lac_con:
    model_copy = copy.deepcopy(model)
    model_copy.reactions.EX_lac__L_e.bounds = (con,1000)
    growth = model_copy.optimize().objective_value
    growth_rates.append(growth)
    yldx = (growth/10)/0.180156
    yld.append(yldx)
    lac = ((con/10)*0.09008)/0.180156 #take the molar yield and convert it to g/g
    lac_g_g.append(lac)
lactate_df = pd.DataFrame({'Lactate production (mol/mol)':lac_con/10, 'lactate yield (g/g)':lac_g_g, 'Growth rate': growth_rates, 'Yield': yld})

In [106]:
lactate_df.to_csv('../databases/lactate_production_table5_fixed.csv')

Then I will do the same, leaving all the production rates free.

In [118]:
model = cobra.io.read_sbml_model('../model/g-thermo.xml')

In [119]:
#make it 'anaerobic'
model.reactions.EX_o2_e.bounds = (-0.1,0)

In [124]:
growth_rates = []
yld = []
lac_g_g =[]
for con in lac_con:
    model_copy = copy.deepcopy(model)
    model_copy.reactions.EX_lac__L_e.bounds = (con,1000)
    growth = model_copy.optimize().objective_value
    growth_rates.append(growth)
    yldx = (growth/10)/0.180156
    yld.append(yldx)
    lac = ((con/10)*0.09008)/0.180156 #take the molar yield and convert it to g/g
    lac_g_g.append(lac)
lactate_df = pd.DataFrame({'Lactate production (mol/mol)':lac_con/10, 'lactate yield (g/g)':lac_g_g, 'Growth rate': growth_rates, 'Yield': yld})

In [125]:
lactate_df.to_csv('../databases/lactate_production_table5_free.csv')